In [36]:
import pprint
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds
import tensorflow_ranking as tfr
import tensorflow_recommenders as tfrs
import keras
import mysql.connector
import pandas as pd
from getpass import getpass
from mysql.connector import connect, Error

# Connect to database
try:
    with connect(
        host="localhost",
        user="root",
        password="mysql",
        database="Sprint1BasicEComDb"
    ) as connection:
        print(connection)
except Error as e:
    print(e)

In [37]:
def get_user_ratings(): #TODO: like get All Books, add attributes to the query
    query = "SELECT userId,bookId,rating FROM UserBookRatings"
    # incase connection is lost, reconnect
    connection.reconnect(attempts=3, delay=5)
    mydb = connection.cursor()
    mydb.execute(query)
    user_ratings = mydb.fetchall()
    return user_ratings


def get_all_books_metadata():
    query =""" 
    """
    # incase connection is lost, reconnect
    connection.reconnect(attempts=3, delay=5)
    mydb = connection.cursor()
    mydb.execute(query)
    all_books = mydb.fetchall()
    return all_books


def get_all_users_metadata():
    query = "SELECT id,birthDate,sex FROM Users"
    # incase connection is lost, reconnect
    connection.reconnect(attempts=3, delay=5)
    mydb = connection.cursor()
    mydb.execute(query)
    all_users = mydb.fetchall()
    return all_users


def save_to_csv(data, filename, header):
    user_ratings_ids = pd.DataFrame(data).set_axis(header, axis=1)
    user_ratings_ids.to_csv(filename, index=False, )
    
def newUserIDs(maxID,quantity):
    newID = []
    for i in range(quantity):
        newID.append(maxID+i+1)
    return newID
    
save_to_csv(get_user_ratings(), 'user_ratings.csv', ['user_id', 'movie_title', 'rating'])
#save_to_csv(get_all_books(), 'all_books.csv', ['book_id', 'book_title', 'description', 'num_pages', 'rating', 'num_of_voters','genres','formats','authors'])
#save_to_csv(get_all_users(), 'all_users.csv', ['user_id', 'birth_date'])



In [38]:
df = get_user_ratings()

user_ratings_ids = pd.DataFrame(df, columns=["user_id", "movie_title", "user_rating"])

users_to_remove = user_ratings_ids['user_id'].drop_duplicates().sample(10)
books_to_remove = user_ratings_ids['movie_title'].drop_duplicates().sample(50)
df_new_users_known_books = user_ratings_ids[~user_ratings_ids['user_id'].isin(users_to_remove) & ~user_ratings_ids['movie_title'].isin(books_to_remove)]
df_old_users_new_books = user_ratings_ids[user_ratings_ids['user_id'].isin(users_to_remove) & ~user_ratings_ids['movie_title'].isin(books_to_remove)]

# convert to string
user_ratings_ids['user_id'] = user_ratings_ids['user_id'].astype(str)
user_ratings_ids['movie_title'] = user_ratings_ids['movie_title'].astype(str)

df_new_users_known_books['user_id'] = df_new_users_known_books['user_id'].astype(str)
df_new_users_known_books['movie_title'] = df_new_users_known_books['movie_title'].astype(str)

df_old_users_new_books['user_id'] = df_old_users_new_books['user_id'].astype(str)
df_old_users_new_books['movie_title'] = df_old_users_new_books['movie_title'].astype(str)

########

rating_rank = user_ratings_ids[['user_id', 'movie_title', 'user_rating']].copy()
book_rank = user_ratings_ids[['movie_title']].copy()

rating_rank10 = df_new_users_known_books[['user_id', 'movie_title', 'user_rating']].copy()
book_rank10 = df_new_users_known_books[['movie_title']].copy()

rating_rank01 = df_old_users_new_books[['user_id', 'movie_title', 'user_rating']].copy()
book_rank01 = df_old_users_new_books[['movie_title']].copy()

user_ratings_ids=[]
df_new_users_known_books=[]
df_old_users_new_books=[]


C:\Users\DZera\AppData\Local\Temp\ipykernel_12596\1318318014.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_new_users_known_books['user_id'] = df_new_users_known_books['user_id'].astype(str)
C:\Users\DZera\AppData\Local\Temp\ipykernel_12596\1318318014.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_new_users_known_books['movie_title'] = df_new_users_known_books['movie_title'].astype(str)
C:\Users\DZera\AppData\Local\Temp\ipykernel_12596\1318318014.py:17: SettingWithCopyWarning: 
A value is 

In [39]:
# Create a tf.data.Dataset from the dataframe
# Will cause error if ran again


books = tf.data.Dataset.from_tensor_slices(dict(book_rank)) # book_rank only 
books = books.map(lambda x: x["movie_title"])

books10 = tf.data.Dataset.from_tensor_slices(dict(book_rank10)) # book_rank only # problem
books10 = books10.map(lambda x: x["movie_title"]) # 

books01 = tf.data.Dataset.from_tensor_slices(dict(book_rank01)) # book_rank only 
books01 = books01.map(lambda x: x["movie_title"])




rating_rank = tf.data.Dataset.from_tensor_slices(dict(rating_rank))
rating_rank = rating_rank.map(lambda x: {
    'user_id': x['user_id'],
    'movie_title': x['movie_title'],
    'user_rating': x['user_rating'],
})

rating_rank10 = tf.data.Dataset.from_tensor_slices(dict(rating_rank10))
rating_rank10 = rating_rank10.map(lambda x: {
    'user_id': x['user_id'],
    'movie_title': x['movie_title'],
    'user_rating': x['user_rating'],
})

rating_rank01 = tf.data.Dataset.from_tensor_slices(dict(rating_rank01))
rating_rank01 = rating_rank01.map(lambda x: {
    'user_id': x['user_id'],
    'movie_title': x['movie_title'],
    'user_rating': x['user_rating'],
})

## new 

unique_books = np.unique(np.concatenate(list(books.batch(1000)))) # is it correct IDK # should be strings
unique_books10 = np.unique(np.concatenate(list(books10.batch(1000)))) # is it correct IDK # should be strings
unique_books01 = np.unique(np.concatenate(list(books01.batch(1000)))) # is it correct IDK # should be strings

unique_user_ids = np.unique(np.concatenate(list(rating_rank.batch(1_000).map(lambda x: x['user_id'])))) ## could be book id
unique_user_ids10 = np.unique(np.concatenate(list(rating_rank10.batch(1_000).map(lambda x: x['user_id']))))
unique_user_ids01 = np.unique(np.concatenate(list(rating_rank01.batch(1_000).map(lambda x: x['user_id']))))

# decode from bytes to string
unique_books = [book.decode('utf-8') for book in unique_books]
unique_user_ids = [user_id.decode('utf-8') for user_id in unique_user_ids]

unique_books10 = [book.decode('utf-8') for book in unique_books10]
unique_user_ids10 = [user_id.decode('utf-8') for user_id in unique_user_ids10]

unique_books01 = [book.decode('utf-8') for book in unique_books01]
unique_user_ids01 = [user_id.decode('utf-8') for user_id in unique_user_ids01]


In [55]:
tf.random.set_seed(42)
# do we have 100_000 ratings? # user_ratings needs to be a new type
shuffled = rating_rank.shuffle(
    100, seed=42, reshuffle_each_iteration=False)

shuffled10 = rating_rank10.shuffle(
    5, seed=42, reshuffle_each_iteration=False)

shuffled01 = rating_rank01.shuffle(
    5, seed=42, reshuffle_each_iteration=False)



train = shuffled.take(70)
test = shuffled.skip(70).take(30)

train10 = shuffled10.take(100)
test10 = shuffled10.skip(3).take(2)
print(len(train10))

train01 = shuffled01.take(10)
test01 = shuffled01.skip(3).take(2)



# We sample 50 lists for each user for the training data. For each list we
# sample 5 movies from the movies the user rated.
train = tfrs.examples.movielens.sample_listwise(
    train,
    num_list_per_user=50,
    num_examples_per_list=5,
    seed=42
)
test = tfrs.examples.movielens.sample_listwise( ## making test empty
    test,
    num_list_per_user=1,
    num_examples_per_list=5,
    seed=42
)

train10 = tfrs.examples.movielens.sample_listwise(
    train10,
    num_list_per_user=100,
    num_examples_per_list=5,
    seed=42
)

test10 = tfrs.examples.movielens.sample_listwise( ## making test empty
    test10,
    num_list_per_user=1,
    num_examples_per_list=5,
    seed=42
)

train01 = tfrs.examples.movielens.sample_listwise(
    train01,
    num_list_per_user=100,
    num_examples_per_list=5,
    seed=42
)

test01 = tfrs.examples.movielens.sample_listwise( ## making test empty
    test01,
    num_list_per_user=1,
    num_examples_per_list=5,
    seed=42
)


print(len(train10))

100
500


In [41]:
class RankingModel(tfrs.Model):

  def __init__(self, loss):
    super().__init__()
    embedding_dimension = 32

    # Compute embeddings for users.
    self.user_embeddings = tf.keras.Sequential([
      tf.keras.layers.StringLookup(
        vocabulary=unique_user_ids),
      tf.keras.layers.Embedding(len(unique_user_ids) + 2, embedding_dimension)
    ])

    # Compute embeddings for movies.
    self.movie_embeddings = tf.keras.Sequential([
      tf.keras.layers.StringLookup(
        vocabulary=unique_books),
      tf.keras.layers.Embedding(len(unique_books) + 2, embedding_dimension)
    ])

    # Compute predictions.
    self.score_model = tf.keras.Sequential([
      # Learn multiple dense layers.
      tf.keras.layers.Dense(256, activation="relu"),
      tf.keras.layers.Dense(64, activation="relu"),
      # Make rating predictions in the final layer.
      tf.keras.layers.Dense(1)
    ])

    self.task = tfrs.tasks.Ranking(
      loss=loss,
      metrics=[
        tfr.keras.metrics.NDCGMetric(name="ndcg_metric"),
        tf.keras.metrics.RootMeanSquaredError()
      ]
    )

  def call(self, features):
    # We first convert the id features into embeddings.
    # User embeddings are a [batch_size, embedding_dim] tensor.
    user_embeddings = self.user_embeddings(features["user_id"])

    # Movie embeddings are a [batch_size, num_movies_in_list, embedding_dim]
    # tensor.
    movie_embeddings = self.movie_embeddings(features["movie_title"])

    # We want to concatenate user embeddings with movie emebeddings to pass
    # them into the ranking model. To do so, we need to reshape the user
    # embeddings to match the shape of movie embeddings.
    list_length = features["movie_title"].shape[1]
    user_embedding_repeated = tf.repeat(
        tf.expand_dims(user_embeddings, 1), [list_length], axis=1)

    # Once reshaped, we concatenate and pass into the dense layers to generate
    # predictions.
    concatenated_embeddings = tf.concat(
        [user_embedding_repeated, movie_embeddings], 2)

    return self.score_model(concatenated_embeddings)

  def compute_loss(self, features, training=False):
    labels = features.pop("user_rating")

    scores = self(features)

    return self.task(
        labels=labels,
        predictions=tf.squeeze(scores, axis=-1),
    )


In [56]:
cached_train = train.shuffle(100).batch(8192).cache()
cached_test = test.batch(4096).cache()

cached_train10 = train10.shuffle(50).batch(8192).cache()
cached_test10 = test10.batch(4096).cache()

print(len(cached_train10))

cached_train01 = train01.shuffle(100).batch(8192).cache()
cached_test01 = test01.batch(4096).cache()

print(len(cached_train01))

1
0


In [57]:
listwise_model = RankingModel(tfr.keras.losses.ListMLELoss())
listwise_model.compile(optimizer=tf.keras.optimizers.Adagrad(0.1))
listwise_model.fit(cached_train, epochs=5, verbose=True)  
listwise_model.fit(cached_train10, epochs=5, verbose=True)  
listwise_model.fit(cached_train01, epochs=5, verbose=True)  

Epoch 1/5
1/1 [==============================] - 5s 5s/step - ndcg_metric: 0.5494 - root_mean_squared_error: 1.5308 - loss: 4.7980 - regularization_loss: 0.0000e+00 - total_loss: 4.7980
Epoch 2/5
1/1 [==============================] - 0s 15ms/step - ndcg_metric: 1.0000 - root_mean_squared_error: 1.5405 - loss: 4.6759 - regularization_loss: 0.0000e+00 - total_loss: 4.6759
Epoch 3/5
1/1 [==============================] - 0s 15ms/step - ndcg_metric: 1.0000 - root_mean_squared_error: 1.5145 - loss: 4.4383 - regularization_loss: 0.0000e+00 - total_loss: 4.4383
Epoch 4/5
1/1 [==============================] - 0s 14ms/step - ndcg_metric: 1.0000 - root_mean_squared_error: 1.3084 - loss: 3.9656 - regularization_loss: 0.0000e+00 - total_loss: 3.9656
Epoch 5/5
1/1 [==============================] - 0s 14ms/step - ndcg_metric: 1.0000 - root_mean_squared_error: 1.1350 - loss: 2.9811 - regularization_loss: 0.0000e+00 - total_loss: 2.9811
Epoch 1/5
1/1 [==============================] - 0s 21ms/step 

ValueError: Expected input data to be non-empty.

: 

In [ ]:
listwise_model_result = listwise_model.evaluate(cached_test, return_dict=True)
print("NDCG of the ListMLE model: {:.4f}".format(listwise_model_result["ndcg_metric"]))
listwise_model.save("easy_listwise_model_saved")


In [ ]:
predictions = listwise_model({
    "user_id": tf.constant(["42"]),
    "movie_title": tf.constant([["1", "2", "3", "4", "5"]])
})

loaded_model = keras.models.load_model('easy_listwise_model_saved')


def make_predictions(model, user_id, books_lists): # model, user_id:str[], books_lists:str[[]], has to be equal lengths
    predictions = []
    for i in range(len(books_lists)):
        predictions.append( model({
            "user_id": tf.constant([user_id[i]]),
            "movie_title": tf.constant([books_lists[i]])
        }))
    return predictions

predictions = make_predictions(listwise_model, ["1000","2"], [["10000", "2", "3", "4", "5"],["1", "2", "3", "4", "5"]])


print(predictions)